In [2]:
# import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import shap
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid


In [7]:
clean_data = pd.read_csv('new_df_womissing.csv')
clean_data

,Unnamed: 0,Account No,Balance,Amount,Third Party Account No,Third Party Name,DateTime
0,0,678330503.0,2971.000000,1584.00,NaN,Westport Care Home,2023-01-01 00:00:00
1,1,472213568.0,3792.000000,1950.00,NaN,Barbiee Boutique,2023-01-01 00:00:00
2,2,472213568.0,3012.000000,-780.00,283027736.0,NaN,2023-01-01 00:00:00
3,3,283027736.0,1787.000000,780.00,472213568.0,NaN,2023-01-01 00:00:00
4,4,624500124.0,3226.000000,1825.00,NaN,Fat Face,2023-01-01 00:00:00
...,...,...,...,...,...,...,...
230121,230121,581655972.0,45935.206861,-41.06,NaN,Tesco,2023-12-06 20:54:00
230122,230122,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local,2023-12-06 20:55:00
230123,230123,824916823.0,9709.172159,-32.94,NaN,Deliveroo,2023-12-06 21:05:00
230124,230124,366550080.0,26834.165794,-19.25,NaN,Amazon,2023-12-06 21:13:00


In [8]:
clean_data.isnull().sum()

Unnamed: 0                     0
Account No                     0
Balance                        0
Amount                         0
Third Party Account No    223297
Third Party Name            6829
DateTime                       0
dtype: int64

In [9]:
clean_data = clean_data.drop(['Unnamed: 0', 'Third Party Account No'], axis=1)

In [13]:
data = clean_data.dropna()
data

,Account No,Balance,Amount,Third Party Name,DateTime
0,678330503.0,2971.000000,1584.00,Westport Care Home,2023-01-01 00:00:00
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,2023-01-01 00:00:00
4,624500124.0,3226.000000,1825.00,Fat Face,2023-01-01 00:00:00
5,203466392.0,4607.660000,2841.66,Lavender Primary,2023-01-01 00:00:00
6,768271776.0,3620.000000,1950.00,A Cut Above,2023-01-01 00:00:00
...,...,...,...,...,...
230121,581655972.0,45935.206861,-41.06,Tesco,2023-12-06 20:54:00
230122,786141370.0,-244.837500,-62.35,Sainsbury Local,2023-12-06 20:55:00
230123,824916823.0,9709.172159,-32.94,Deliveroo,2023-12-06 21:05:00
230124,366550080.0,26834.165794,-19.25,Amazon,2023-12-06 21:13:00


In [14]:
data['Third Party Name'].unique()

array(['Westport Care Home', 'Barbiee Boutique', 'Fat Face',
       'Lavender Primary', 'A Cut Above', 'Tesco', 'Brilliant Brushes',
       'Costa Coffee', 'Premier Finance', 'North Face', 'Craftastic',
       'Fitted Stitch', 'Selfridges', 'Starbucks', 'The Crown',
       'Cass Art', 'Green Park Academy', 'Rose & Crown', 'Kings Arms',
       'Sunny Care Nursery', 'A Yarn Story', 'Pets Corner',
       'Lavender Fields', 'Victoria Park', 'Halifax', 'LBG', 'Blizzard',
       'Xbox', 'Mojang Studios', 'PureGym', 'Disney', 'Netflix',
       'Grand Union BJJ', 'Amazon', 'SquareOnix', 'Deliveroo', 'JustEat',
       'Coop Local', 'Sainsbury', 'AMAZON', 'The Works', "Blackwell's",
       'Topshop', 'Matalan', 'Foyles', 'Wool', 'Hobby Lobby', 'Revella',
       'Sainsbury Local', 'Loosely Fitted', 'Stitch By Stitch',
       'Coffee #1', 'Hobbycraft', 'Lloyds Pharmacy', 'Gamestation', 'CeX',
       'Etsy', 'Five Senses Art', 'Sports Direct', 'Boots', 'Reebok',
       'JD Sports', 'Head', 'Frankie

In [15]:
holiday_data = {
    'date': ['2025-01-01', '2025-04-18', '2025-12-25', '2025-12-26', '2025-04-18', '2025-04-21', '2025-05-05', '2025-05-26', '2025-08-25'],
    'holiday': ['New Year\'s Day', 'Good Friday', 'Christmas Day', 'Boxing Day', 'Good Friday', 'Easter Monday', 'Early May bank holiday', 'Spring bank holiday', 'Summer bank holiday']
}
df_holidays = pd.DataFrame(holiday_data)
df_holidays

,date,holiday
0,2025-01-01,New Year's Day
1,2025-04-18,Good Friday
2,2025-12-25,Christmas Day
3,2025-12-26,Boxing Day
4,2025-04-18,Good Friday
5,2025-04-21,Easter Monday
6,2025-05-05,Early May bank holiday
7,2025-05-26,Spring bank holiday
8,2025-08-25,Summer bank holiday


In [16]:
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_holidays['month_day'] = df_holidays['date'].dt.strftime('%m-%d')

data['DateTime'] = pd.to_datetime(data['DateTime'])
data['month_day'] = data['DateTime'].dt.strftime('%m-%d')

df_holidays = df_holidays.drop_duplicates(subset=['month_day'])

data['Holiday'] = data['month_day'].map(df_holidays.set_index('month_day')['holiday'])

data


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Account No,Balance,Amount,Third Party Name,DateTime,month_day,Holiday
0,678330503.0,2971.000000,1584.00,Westport Care Home,2023-01-01 00:00:00,01-01,New Year's Day
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,2023-01-01 00:00:00,01-01,New Year's Day
4,624500124.0,3226.000000,1825.00,Fat Face,2023-01-01 00:00:00,01-01,New Year's Day
5,203466392.0,4607.660000,2841.66,Lavender Primary,2023-01-01 00:00:00,01-01,New Year's Day
6,768271776.0,3620.000000,1950.00,A Cut Above,2023-01-01 00:00:00,01-01,New Year's Day
...,...,...,...,...,...,...,...
230121,581655972.0,45935.206861,-41.06,Tesco,2023-12-06 20:54:00,12-06,NaN
230122,786141370.0,-244.837500,-62.35,Sainsbury Local,2023-12-06 20:55:00,12-06,NaN
230123,824916823.0,9709.172159,-32.94,Deliveroo,2023-12-06 21:05:00,12-06,NaN
230124,366550080.0,26834.165794,-19.25,Amazon,2023-12-06 21:13:00,12-06,NaN


In [17]:
data.fillna(False)

,Account No,Balance,Amount,Third Party Name,DateTime,month_day,Holiday
0,678330503.0,2971.000000,1584.00,Westport Care Home,2023-01-01 00:00:00,01-01,New Year's Day
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,2023-01-01 00:00:00,01-01,New Year's Day
4,624500124.0,3226.000000,1825.00,Fat Face,2023-01-01 00:00:00,01-01,New Year's Day
5,203466392.0,4607.660000,2841.66,Lavender Primary,2023-01-01 00:00:00,01-01,New Year's Day
6,768271776.0,3620.000000,1950.00,A Cut Above,2023-01-01 00:00:00,01-01,New Year's Day
...,...,...,...,...,...,...,...
230121,581655972.0,45935.206861,-41.06,Tesco,2023-12-06 20:54:00,12-06,False
230122,786141370.0,-244.837500,-62.35,Sainsbury Local,2023-12-06 20:55:00,12-06,False
230123,824916823.0,9709.172159,-32.94,Deliveroo,2023-12-06 21:05:00,12-06,False
230124,366550080.0,26834.165794,-19.25,Amazon,2023-12-06 21:13:00,12-06,False


In [18]:
def check_holiday(holiday):
    if isinstance(holiday, str):
        return True
    else:
        return False

# Apply the function to the 'Holiday' column
data['Holiday'] = data['Holiday'].apply(check_holiday)

data


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Account No,Balance,Amount,Third Party Name,DateTime,month_day,Holiday
0,678330503.0,2971.000000,1584.00,Westport Care Home,2023-01-01 00:00:00,01-01,True
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,2023-01-01 00:00:00,01-01,True
4,624500124.0,3226.000000,1825.00,Fat Face,2023-01-01 00:00:00,01-01,True
5,203466392.0,4607.660000,2841.66,Lavender Primary,2023-01-01 00:00:00,01-01,True
6,768271776.0,3620.000000,1950.00,A Cut Above,2023-01-01 00:00:00,01-01,True
...,...,...,...,...,...,...,...
230121,581655972.0,45935.206861,-41.06,Tesco,2023-12-06 20:54:00,12-06,False
230122,786141370.0,-244.837500,-62.35,Sainsbury Local,2023-12-06 20:55:00,12-06,False
230123,824916823.0,9709.172159,-32.94,Deliveroo,2023-12-06 21:05:00,12-06,False
230124,366550080.0,26834.165794,-19.25,Amazon,2023-12-06 21:13:00,12-06,False


In [19]:
data

,Account No,Balance,Amount,Third Party Name,DateTime,month_day,Holiday
0,678330503.0,2971.000000,1584.00,Westport Care Home,2023-01-01 00:00:00,01-01,True
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,2023-01-01 00:00:00,01-01,True
4,624500124.0,3226.000000,1825.00,Fat Face,2023-01-01 00:00:00,01-01,True
5,203466392.0,4607.660000,2841.66,Lavender Primary,2023-01-01 00:00:00,01-01,True
6,768271776.0,3620.000000,1950.00,A Cut Above,2023-01-01 00:00:00,01-01,True
...,...,...,...,...,...,...,...
230121,581655972.0,45935.206861,-41.06,Tesco,2023-12-06 20:54:00,12-06,False
230122,786141370.0,-244.837500,-62.35,Sainsbury Local,2023-12-06 20:55:00,12-06,False
230123,824916823.0,9709.172159,-32.94,Deliveroo,2023-12-06 21:05:00,12-06,False
230124,366550080.0,26834.165794,-19.25,Amazon,2023-12-06 21:13:00,12-06,False


In [20]:
RANDOM_STATE = 10
SALARY_DAY = 1

# convert 'DateTime' to datetime and extract features
data['DateTime'] = pd.to_datetime(data['DateTime'])
data['Hour'] = data['DateTime'].dt.hour
data['DayOfWeek'] = data['DateTime'].dt.dayofweek
data['Day'] = data['DateTime'].dt.day

data['Is_Salary'] = (data['Day'] == SALARY_DAY)

# drop the original 'DateTime' column
data = data.drop(['DateTime'], axis=1)

X = data.drop(['Account No'], axis=1)
X



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,Balance,Amount,Third Party Name,month_day,Holiday,Hour,DayOfWeek,Day,Is_Salary
0,2971.000000,1584.00,Westport Care Home,01-01,True,0,6,1,True
1,3792.000000,1950.00,Barbiee Boutique,01-01,True,0,6,1,True
4,3226.000000,1825.00,Fat Face,01-01,True,0,6,1,True
5,4607.660000,2841.66,Lavender Primary,01-01,True,0,6,1,True
6,3620.000000,1950.00,A Cut Above,01-01,True,0,6,1,True
...,...,...,...,...,...,...,...,...,...
230121,45935.206861,-41.06,Tesco,12-06,False,20,2,6,False
230122,-244.837500,-62.35,Sainsbury Local,12-06,False,20,2,6,False
230123,9709.172159,-32.94,Deliveroo,12-06,False,21,2,6,False
230124,26834.165794,-19.25,Amazon,12-06,False,21,2,6,False


In [23]:
X_drop_test = data.drop(['Account No', 'Holiday', 'Is_Salary', 'month_day'], axis=1)
X_drop_test

,Balance,Amount,Third Party Name,Hour,DayOfWeek,Day
0,2971.000000,1584.00,Westport Care Home,0,6,1
1,3792.000000,1950.00,Barbiee Boutique,0,6,1
4,3226.000000,1825.00,Fat Face,0,6,1
5,4607.660000,2841.66,Lavender Primary,0,6,1
6,3620.000000,1950.00,A Cut Above,0,6,1
...,...,...,...,...,...,...
230121,45935.206861,-41.06,Tesco,20,2,6
230122,-244.837500,-62.35,Sainsbury Local,20,2,6
230123,9709.172159,-32.94,Deliveroo,21,2,6
230124,26834.165794,-19.25,Amazon,21,2,6


# Without flag columns

In [24]:
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ['Third Party Name']),
        ('scaler', StandardScaler(), ['Balance', 'Amount', 'Hour', 'DayOfWeek'])
    ],
    remainder='passthrough'
)


X_preprocessed = column_transformer.fit_transform(X_drop_test)

X_train, X_val = train_test_split(X_preprocessed, test_size=0.2, random_state=RANDOM_STATE)

# One-Class SVM parem
param_grid = {
    'nu': [0.01, 0.05, 0.1],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['auto', 'scale']
}

# find bes param
best_score = np.inf
best_params = None

for g in ParameterGrid(param_grid):
    model = OneClassSVM(**g)
    model.fit(X_train)
    scores = model.decision_function(X_val)
    score = -scores.mean()
    if score < best_score:
        best_score = score
        best_params = g

print("Best parameters:", best_params)

oc_svm_best = OneClassSVM(**best_params)




Best parameters: {'gamma': 'auto', 'kernel': 'sigmoid', 'nu': 0.1}


In [26]:
oc_svm_best.fit(X_preprocessed)

anomaly_scores = oc_svm_best.decision_function(X_preprocessed)

In [35]:
X_drop_test

,Balance,Amount,Third Party Name,Hour,DayOfWeek,Day
0,2971.000000,1584.00,Westport Care Home,0,6,1
1,3792.000000,1950.00,Barbiee Boutique,0,6,1
4,3226.000000,1825.00,Fat Face,0,6,1
5,4607.660000,2841.66,Lavender Primary,0,6,1
6,3620.000000,1950.00,A Cut Above,0,6,1
...,...,...,...,...,...,...
230121,45935.206861,-41.06,Tesco,20,2,6
230122,-244.837500,-62.35,Sainsbury Local,20,2,6
230123,9709.172159,-32.94,Deliveroo,21,2,6
230124,26834.165794,-19.25,Amazon,21,2,6


In [27]:
anomaly_scores

array([ 21.76159437,  18.31762344,  29.9359342 , ..., 892.19723642,
       846.44409515, 912.75977171])

In [28]:
anomaly_scores_df = pd.DataFrame(anomaly_scores, columns=['Anomaly Score'])

data.reset_index(drop=True, inplace=True)

# concatenate the anomaly scores with the original data
data_with_scores = pd.concat([data, anomaly_scores_df], axis=1)

print(data_with_scores.head())

    Account No  Balance   Amount    Third Party Name month_day  Holiday  Hour  \
0  678330503.0  2971.00  1584.00  Westport Care Home     01-01     True     0   
1  472213568.0  3792.00  1950.00    Barbiee Boutique     01-01     True     0   
2  624500124.0  3226.00  1825.00            Fat Face     01-01     True     0   
3  203466392.0  4607.66  2841.66    Lavender Primary     01-01     True     0   
4  768271776.0  3620.00  1950.00         A Cut Above     01-01     True     0   

   DayOfWeek  Day  Is_Salary  Anomaly Score  
0          6    1       True      21.761594  
1          6    1       True      18.317623  
2          6    1       True      29.935934  
3          6    1       True       7.685006  
4          6    1       True      19.016573  


In [29]:
neg = data_with_scores[data_with_scores['Anomaly Score'] < 0]
neg

,Account No,Balance,Amount,Third Party Name,month_day,Holiday,Hour,DayOfWeek,Day,Is_Salary,Anomaly Score
8,832564361.0,5376.000000,3542.00,Premier Finance,01-01,True,0,6,1,True,-13.132777
22,298000165.0,6040.000000,4167.00,Selfridges,01-01,True,0,6,1,True,-39.365907
27,183888402.0,4966.000000,4167.00,Fitted Stitch,01-01,True,0,6,1,True,-36.485650
42,705848712.0,6390.800000,5292.80,Green Park Academy,01-01,True,0,6,1,True,-111.345680
100,131388684.0,5225.000000,4375.00,Premier Finance,01-01,True,0,6,1,True,-49.580492
...,...,...,...,...,...,...,...,...,...,...,...
220914,331450202.0,9626.389441,-16.90,JustEat,12-02,False,20,5,2,False,-244.956540
220915,553734946.0,698.208600,-78.00,Sainsbury,12-02,False,20,5,2,False,-227.013554
220916,679662108.0,18453.801150,-5.47,Revella,12-02,False,21,5,2,False,-291.649002
220917,549414742.0,8397.777374,-24.18,Deliveroo,12-02,False,21,5,2,False,-268.302817


In [30]:
data_with_scores[data_with_scores['Anomaly Score'] >= 0]


,Account No,Balance,Amount,Third Party Name,month_day,Holiday,Hour,DayOfWeek,Day,Is_Salary,Anomaly Score
0,678330503.0,2971.000000,1584.00,Westport Care Home,01-01,True,0,6,1,True,21.761594
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,01-01,True,0,6,1,True,18.317623
2,624500124.0,3226.000000,1825.00,Fat Face,01-01,True,0,6,1,True,29.935934
3,203466392.0,4607.660000,2841.66,Lavender Primary,01-01,True,0,6,1,True,7.685006
4,768271776.0,3620.000000,1950.00,A Cut Above,01-01,True,0,6,1,True,19.016573
...,...,...,...,...,...,...,...,...,...,...,...
223292,581655972.0,45935.206861,-41.06,Tesco,12-06,False,20,2,6,False,831.089592
223293,786141370.0,-244.837500,-62.35,Sainsbury Local,12-06,False,20,2,6,False,939.216655
223294,824916823.0,9709.172159,-32.94,Deliveroo,12-06,False,21,2,6,False,892.197236
223295,366550080.0,26834.165794,-19.25,Amazon,12-06,False,21,2,6,False,846.444095


In [31]:
# find threshold of anomaly scores
threshold = np.percentile(anomaly_scores, 1)
threshold

-417.43508750429476

In [32]:
# use threshold to identifind the anomaly behavior
data_with_scores['Is_Anomaly'] = anomaly_scores < threshold

data_with_scores

,Account No,Balance,Amount,Third Party Name,month_day,Holiday,Hour,DayOfWeek,Day,Is_Salary,Anomaly Score,Is_Anomaly
0,678330503.0,2971.000000,1584.00,Westport Care Home,01-01,True,0,6,1,True,21.761594,False
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,01-01,True,0,6,1,True,18.317623,False
2,624500124.0,3226.000000,1825.00,Fat Face,01-01,True,0,6,1,True,29.935934,False
3,203466392.0,4607.660000,2841.66,Lavender Primary,01-01,True,0,6,1,True,7.685006,False
4,768271776.0,3620.000000,1950.00,A Cut Above,01-01,True,0,6,1,True,19.016573,False
...,...,...,...,...,...,...,...,...,...,...,...,...
223292,581655972.0,45935.206861,-41.06,Tesco,12-06,False,20,2,6,False,831.089592,False
223293,786141370.0,-244.837500,-62.35,Sainsbury Local,12-06,False,20,2,6,False,939.216655,False
223294,824916823.0,9709.172159,-32.94,Deliveroo,12-06,False,21,2,6,False,892.197236,False
223295,366550080.0,26834.165794,-19.25,Amazon,12-06,False,21,2,6,False,846.444095,False


In [33]:
neg[neg['Account No']==472213568.0]

,Account No,Balance,Amount,Third Party Name,month_day,Holiday,Hour,DayOfWeek,Day,Is_Salary,Anomaly Score
801,472213568.0,2404.010000,-7.99,Disney,01-01,True,0,6,1,True,-1.807850
802,472213568.0,2388.020000,-15.99,Netflix,01-01,True,0,6,1,True,-1.238013
803,472213568.0,2382.030000,-5.99,Mojang Studios,01-01,True,0,6,1,True,-1.553827
804,472213568.0,2375.030000,-7.00,Xbox,01-01,True,0,6,1,True,-1.502786
805,472213568.0,2275.030000,-100.00,Grand Union BJJ,01-01,True,0,6,1,True,-2.959278
4513,472213568.0,2240.070000,-34.96,The Works,01-01,True,9,6,1,True,-226.136721
21290,472213568.0,3266.314181,1950.00,Barbiee Boutique,02-01,False,0,2,1,True,-32.062270
21768,472213568.0,1886.314181,-600.00,LBG,02-01,False,0,2,1,True,-39.027327
26619,472213568.0,1740.777190,-8.57,Lloyds Pharmacy,02-02,False,18,3,2,False,-202.805268
38662,472213568.0,2891.848897,1950.00,Barbiee Boutique,03-01,False,0,2,1,True,-31.084538


# With flag column

In [25]:
X = data.drop(['month_day'], axis=1)
X


,Account No,Balance,Amount,Third Party Name,Holiday,Hour,DayOfWeek,Day,Is_Salary
0,678330503.0,2971.000000,1584.00,Westport Care Home,True,0,6,1,True
1,472213568.0,3792.000000,1950.00,Barbiee Boutique,True,0,6,1,True
4,624500124.0,3226.000000,1825.00,Fat Face,True,0,6,1,True
5,203466392.0,4607.660000,2841.66,Lavender Primary,True,0,6,1,True
6,768271776.0,3620.000000,1950.00,A Cut Above,True,0,6,1,True
...,...,...,...,...,...,...,...,...,...
230121,581655972.0,45935.206861,-41.06,Tesco,False,20,2,6,False
230122,786141370.0,-244.837500,-62.35,Sainsbury Local,False,20,2,6,False
230123,824916823.0,9709.172159,-32.94,Deliveroo,False,21,2,6,False
230124,366550080.0,26834.165794,-19.25,Amazon,False,21,2,6,False


In [25]:
RANDOM_STATE = 10
SALARY_DAY = 1

column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ['Third Party Name']),
        ('scaler', StandardScaler(), ['Balance', 'Amount', 'Hour', 'DayOfWeek'])
    ],
    remainder='passthrough'
)


X_preprocessed = column_transformer.fit_transform(X)

X_train, X_val = train_test_split(X_preprocessed, test_size=0.2, random_state=RANDOM_STATE)

# One-Class SVM parem
param_grid = {
    'nu': [0.01, 0.05, 0.1],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['auto', 'scale']
}

# find bes param
best_score = np.inf
best_params = None

for g in ParameterGrid(param_grid):
    model = OneClassSVM(**g)
    model.fit(X_train)
    scores = model.decision_function(X_val)
    score = -scores.mean()
    if score < best_score:
        best_score = score
        best_params = g

print("Best parameters:", best_params)

oc_svm_best = OneClassSVM(**best_params)


In [ ]:
oc_svm_best.fit(X_preprocessed)

anomaly_scores = oc_svm_best.decision_function(X_preprocessed)


In [ ]:
anomaly_scores

In [ ]:
anomaly_scores_df = pd.DataFrame(anomaly_scores, columns=['Anomaly Score'])

data.reset_index(drop=True, inplace=True)

# concatenate the anomaly scores with the original data
data_with_scores = pd.concat([data, anomaly_scores_df], axis=1)

print(data_with_scores.head())

In [ ]:
neg = data_with_scores[data_with_scores['Anomaly Score'] < 0]
neg


In [ ]:
data_with_scores[data_with_scores['Anomaly Score'] >= 0]


In [ ]:
plt.hist(data_with_scores['Anomaly Score'], bins=50)
plt.title('Distribution of Anomaly Scores')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# find threshold of anomaly scores
threshold = np.percentile(anomaly_scores, 1)
threshold


In [ ]:
# use threshold to identifind the anomaly behavior
data_with_scores['Is_Anomaly'] = anomaly_scores < threshold

data_with_scores


In [ ]:
neg[neg['Account No']==472213568.0]

# it is big manitude minus if people get salary every month

In [ ]:
data_with_scores

In [ ]:
# show stats of 'Third Party Name'
transaction_stats = data_with_scores.groupby('Account No').agg({
    'Amount': ['mean', 'std', 'count', 'sum'],
    'Anomaly Score': ['mean', 'min', 'max']
}).reset_index()

transaction_stats


In [ ]:
# identify the rik of 'Anomaly Score'
transaction_stats['Risk Score'] = -transaction_stats['Anomaly Score', 'mean']
transaction_stats['Medium Score']  = -transaction_stats['Anomaly Score', 'mean']
transaction_stats['Low Score']  = -transaction_stats['Anomaly Score', 'mean']


risk_threshold = transaction_stats['Risk Score'].quantile(0.95)
risk_threshold_2 = transaction_stats['Risk Score'].quantile(0.1)

transaction_stats['Is_High_Risk'] = transaction_stats['Risk Score'] > risk_threshold
transaction_stats['Is_Medium_Risk'] = ((transaction_stats['Medium Score'] < risk_threshold) & (transaction_stats['Medium Score'] > risk_threshold_2))
transaction_stats['Is_Low_Risk'] = transaction_stats['Low Score'] < risk_threshold_2

high_risk_accounts = transaction_stats[transaction_stats['Is_High_Risk']]
high_risk_accounts


In [ ]:
high_risk_accounts.shape

In [ ]:
md_risk_accounts = transaction_stats[transaction_stats['Is_Medium_Risk']]
md_risk_accounts

In [ ]:
low_risk_accounts = transaction_stats[transaction_stats['Is_Low_Risk']]
low_risk_accounts